<header style="padding:10px;background:#f9f9f9;border-top:3px solid #00b2b1">
    <h3> Machine Learning Analytics (MLA) with Teradata Vantage</h3>
    <h2>Python Basics: Managing Database Tables</h2>
    <p>This notebook covers how to manage permissions of database tables using Python teradataml. To create tables and import data, see "PythonBasics-2-DataTransfer.ipynb" to view the various ways to import data to create tables.</p>
    <p> In this notebook, we will demonstrate the following:</p>
    <ul><li>a simple example of creating a table</li>
        <li>how to GRANT permissions to the table</li>
        <li>how to list tables (or views)</li>
        <li>how to delete the table (or view)</li>
    </ul>
    <p>For reference documentation, go to <a href="https://docs.teradata.com">https://docs.teradata.com</a> and search for "Teradata Package for Python Function  Reference".</p>
    <p>For more information on Machine Learning Analytics (MLA), go to the <a href="https://uhgazure.sharepoint.com/sites/UDW/SitePages/Machine-Learning-Analytics.aspx" target="new">Machine Learning Analytics (MLA) Sharepoint Site</a>.
</header>

#### Install teradataml package
Note: You only need to run this once. The "!" allows you to run Linux script from the notebook cell. 

In [ ]:
!pip install teradataml --user

#### Import Packages

In [1]:
# to hide authentication strings
import getpass as gp

# for managing connections
from teradataml import create_context, get_context, remove_context

# for setting configure options
from teradataml import configure

# for managing DataFrames
from teradataml.dataframe.dataframe import DataFrame

# for dropping or viewint tables and views
from teradataml.dbutils.dbutils import db_drop_table, db_drop_view, db_list_tables

# for defining SQL data types
from teradatasqlalchemy.types import *

import pandas as pd
import numpy as np

### Connection Variables

##### Set User and Password Variables

In [ ]:
user = gp.getpass("User")

In [ ]:
password = gp.getpass("Password")

##### Set Connection Variables

In [2]:
host = 'UDWProd.uhc.com'
logmech = 'LDAP'
defaultDB = 'MLA_XXX'  # use your MLA database (MLA_BOPS, MLA_CNS, MLA_ENI, MLA_MNR)

##### Virtual DataFrames are only allowed in the MLA Databases.

If you are using a default database other than database="MLA_xxx", you must set the configure options below to your MLA database. The MLA database is where your Virtual DataFrames will be written.

<font color="red">Note: you only need to run this cell below if your default database is NOT MLA_XXX database.</font>

In [ ]:
mlaDB = "MLA_XXX" # use your MLA database (MLA_BOPS, MLA_CNS, MLA_ENI, MLA_MNR)

# this is the MLA database to which teradataml virtual tables will be written. 
configure.temp_table_database = defaultDB if defaultDB[:3]=="MLA" else mlaDB

# this is the MLA database to which teradataml virtual views will be written.
configure.temp_view_database = defaultDB if defaultDB[:3]=="MLA" else mlaDB 

##### Create Context
See the PythonBasics-1-ConnectingToVantage Notebook for more information about contexts and garbage collection.  

In [4]:
td_context = create_context(host = host, 
                            username= user, 
                            password = password, 
                            logmech='LDAP', 
                            sslmode='ALLOW', 
                            database=defaultDB)

### Load Packages

##### Load CSV file locally into Pandas:  Surveys

In [5]:
surveys = pd.read_csv('data/surveysdata.csv', 
                      names=["record_id","month","day","year","plot_id","species_id","sex","hindfoot_length","weight"])
surveys.head()

,record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight
0,12552,4,6,1987,4,RM,F,16.0,9.0
1,26704,7,30,1997,8,PP,M,20.0,14.0
2,4434,5,4,1981,9,DM,F,36.0,44.0
3,24153,6,14,1996,4,DM,M,38.0,46.0
4,5,7,16,1977,3,DM,M,35.0,NaN


In [6]:
DataFrame.to_sql(surveys, table_name=f'{user}_surveys_1', if_exists='replace')

#### Check if table was created using the db_list_tables() function

`db_list_tables(schema_name=None, object_name=None, object_type='all')`

This function lists the Vantage object names for tables and/or views for the specified schema name. If none is included, it will use the default database.

In [9]:
import warnings
warnings.filterwarnings("ignore")
db_list_tables(schema_name=defaultDB, object_name="%_surveys_%")

,TableName
0,tlugtu_surveys_1


#### Granting Permissions
When you create a table, other users of the database don't necessarily have access to it. But if you are creating a table for a team project that others must be able to access, you can GRANT them permissions by username or role. To run these scripts, you can execute a SQL script from the connect() object of the create_context() object. 

In [10]:
# Set the table, user, and/or roles that will be used for this exercise

targetObject = f'{user}_surveys_1' # this is the table that we just created
userToGrant = 'jsumme17' # this is Jim Summersgill on the Teradata/BPMO Team
roleToGrant = f'P_{defaultDB.upper()}_USER_ROLE' # this is the end user role of everyone that has access to this database 

print(f"Target Object:  {targetObject}")
print(f"User Name:  {userToGrant}")
print(f"Role Name:  {roleToGrant}")

Target Object:  tlugtu_surveys_1
User Name:  jsumme17
Role Name:  P_MLA_BOPS_USER_ROLE


##### Get the connect object

In [11]:
conn = td_context.connect()

##### Apply permissions for a table or view to a USER with the .execute() method

The format of granting is as follows:
`GRANT SELECT ON [schema].[NewTableOrViewName] TO [username]`

In [12]:
# Example for granting select permissions to an individual user
SQLstr = f"GRANT SELECT ON {defaultDB}.{targetObject} TO {userToGrant}";
print(SQLstr)
conn.execute(SQLstr)

GRANT SELECT ON MLA_BOPS.tlugtu_surveys_1 TO jsumme17


##### Apply permissions for a table or view to a ROLE with the .execute() method

The format of granting is as follows:
`GRANT SELECT ON [schema].[NewTableOrViewName] TO [rolename]`

In [13]:
# Example for granting select permissions to a role
SQLstr = f"GRANT SELECT ON {defaultDB}.{targetObject} TO {roleToGrant}";
print(SQLstr)
conn.execute(SQLstr)

GRANT SELECT ON MLA_BOPS.tlugtu_surveys_1 TO P_MLA_BOPS_USER_ROLE


#### Dropping a table or view

`db_drop_table(table_name, schema_name=None)`
Drops the table from a given schema

`db_drop_view(view_name, schema_name=None)`
Drops the view from a given schema

In [14]:
try:
    db_drop_table(targetObject, schema_name=defaultDB)
except:
    pass

In [15]:
remove_context()

True

<span style="font-size:16px;">For online documentation on Teradata Vantage analytic functions, refer to the [Teradata Developer Portal](https://docs.teradata.com/) and search for phrases "Python User Guide" and "Python Function Reference".</span>